In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

np.random.seed(163)

%run ./DiffKF.ipynb
%run ./KalmanFilter.ipynb

In [ ]:
dt = 10
q = 0.001

C = np.array([[0.34,0.33, 0, 0, 0.33],[0.33,0.34,0.33,0,0],[0,0.33,0.34,0.33,0],[0,0,0.33,0.34,0.33],[0.33,0,0,0.33,0.34]])
m = len(C[0])

A = np.array([[1, dt, 0, 0], [0, 1, 0, 0],[0,0,1,dt], [0, 0, 0, 1]])
H = np.array([[1, 0, 0, 0],[0,0,1,0]])

Q = q*np.array([[(dt**3)/3, (dt**2)/2, 0, 0], [(dt**2)/2, dt, 0, 0],[0,0,(dt**3)/3,(dt**2)/2], [0, 0, (dt**2)/2, dt]])
R = np.array([[4,0],[0,4]])

A_kf = np.kron(np.eye(m),A)
H_kf = np.kron(np.eye(m),H)
Q_kf = np.kron(np.eye(m),Q)
R_kf = np.kron(np.eye(m),R)

kf_state_size = A_kf.shape[0]
kf_measure_size = R_kf.shape[0]

F = [A for _ in range(n)]
G = [np.eye(4) for _ in range(n)]
H_n = [H for _ in range(n)]

Q_dkf = [Q for _ in range(n)]
R_dkf = [R for _ in range(n)]

In [ ]:
procc_noise_kf = lambda : np.linalg.cholesky(Q_kf) @ np.random.normal(np.array([[0 for _ in range(kf_state_size)]]).T)
measure_noise_kf = lambda : np.linalg.cholesky(R_kf) @ np.random.normal(np.array([[0 for _ in range(kf_measure_size)]]).T)

procc_noise_dkf = lambda : np.linalg.cholesky(Q_dkf) @ np.random.normal(np.array([[0,0,0,0]]).T)
measure_noise_dkf = lambda : np.linalg.cholesky(R_dkf) @ np.random.normal(np.array([[0,0]]).T)

In [ ]:

n = 40

#True Initial
x0 = np.array([[np.random.normal(0,2) for _ in range(A_kf.shape[0])]]).T

#Initial Estimate
x = x0 + procc_noise_kf()
P = 10*np.eye(kf_state_size)

kf = KalmanFilter(A = A,H = H, Q = Q, R = R,P=P,x0=x)

In [ ]:
P_n = [10*np.eye(state_size) for _ in range(m)]


DKF = DiffKF(C,F,G,H_n,R_n,Q_n,x0,P_n)

z = [(H[0] @ xp)+measure_noise(i) for i in range(n)]

truth = np.zeros((iters+1,m,1))
truth[0] = xp+procc_noise(0)
predictions = {i:[] for i in range(n)}
measurements = [z]
errors = np.zeros((iters,n,m,1))
P_hist = np.zeros((iters,n,m,m))

In [ ]:
for i in range(iters):
    x = DKF.predict()
    z = [(H[0] @ truth[i])+measure_noise(j) for j in range(n)]
    truth[i+1] = F[0]@truth[i]+procc_noise(0)
    measurements.append(z)
    errors[i] = np.array([node.x-truth[i-1] for node in DKF.nodes])
    P_hist[i] = np.array([node.P for node in DKF.nodes])
    DKF.update(z)
    for j in range(n):
        predictions[j].append(H[j]@DKF.nodes[j].x)

In [ ]:
procc_noise = lambda : np.linalg.cholesky(Q) @ np.random.normal(np.array([[0,0,0,0]]).T)
measure_noise = lambda : np.linalg.cholesky(R) @ np.random.normal(np.array([[0,0]]).T)


truth = [(A@x0)+procc_noise()]
# truth = [(A@x0)]
measurements = [(H @ x0)+measure_noise()]

for i in range(n):
    measurements.append(np.dot(H,truth[-1])+measure_noise())
    truth.append(A@truth[-1]+procc_noise())
    # truth.append(A@truth[-1])

predictions = []
errors = np.zeros((n,4,1))
P_hist = np.zeros((n,4,4))

for i in range(1,n):
    x = H @ kf.predict()
    errors[i]=kf.x-truth[i-1]
    P_hist[i]=(kf.P)
    K = kf.update(measurements[i])
    predictions.append(H@(kf.x))